In [1]:
import pandas as pd
import altair as alt
from altair import expr, datum


In [2]:
!pip install -U altair

  Using cached https://files.pythonhosted.org/packages/01/55/0bb2226e34f21fa549c3f4557b4f154a5632f61132a969da17c95ca8eab9/altair-4.1.0-py3-none-any.whl
  Found existing installation: altair 4.0.0
    Uninstalling altair-4.0.0:
      Successfully uninstalled altair-4.0.0


In [2]:
def select_range(name: str, default, min, max, step):
    selection = alt.selection_single(
        bind=alt.binding_range(min=min, max=max, step=step, name=name),
        fields=['_'],
        init={'_': default}
    )
    return selection, selection._

In [3]:
lease_term_selection, lease_term = select_range(
    'Lease Term', 30, 0, 45, 5
)
# https://patch.com/massachusetts/boston/2020-residential-property-tax-rates-344-ma-communities
interest_selection, interest = select_range(
    'Interest Rate', 3, 0.1, 10, 0.1
)
# https://www.nerdwallet.com/blog/mortgages/the-real-cost-of-your-house/
savings_selection, savings = select_range(
    'Savings (annually, % of home value)', 2, 0, 20, 1
)
tax_selection, tax = select_range(
    'Tax Rate', 1.5, 1.0, 2.2, 0.1
)
# https://www.bankrate.com/insurance/homeowners-insurance/homeowners-insurance-cost/
insurance_selection, insurance = select_range(
    'Insurance (yearly, for 4 rooms)', 1700, 1200, 2200, 10
)
utilities_selection, utilities  = select_range(
    'Uilities (monthly, per Room)', 50, 0, 100, 0.1
)
# https://www.nerdwallet.com/article/mortgages/closing-costs-mortgage-fees-explained
# 2-5%
closing_fee_selection, closing_fee = select_range(
    'Closing Fee Rate', 3, 0, 5, 1
)
rooms_selection, rooms = select_range(
    'Rooms', 4, 1, 7, 1
)

cost = datum.cost


loan_amount = cost + expr.round(cost * closing_fee / 100)
monthly_interest = interest / 12 / 100
n_months = lease_term * 12

calculations = {
    "mortgage_payment": expr.round(expr.if_(
        lease_term <= 0,
        # interest only
        monthly_interest * loan_amount,
        monthly_interest * loan_amount / (1 - ((1 + monthly_interest) ** (-n_months)))
    )),
    "savings_month":  expr.round(cost * (savings / 100) / 12),
    "tax_month":  expr.round(cost * tax / 100 / 12),
    "insurance_month":  expr.round(insurance / 12),
    "utilities": expr.round(utilities),
    "total_monthly": (
        datum.mortgage_payment +
        datum.savings_month +
        datum.tax_month +
        datum.insurance_month + 
        datum.utilities
    ),
    "rent": expr.round(datum.total_monthly / rooms)
}

# TODO: add PMI

In [5]:
MAX_COST = 500
bars = alt.Chart(
    alt.sequence(
        60 * 1000,
        500 * 1000,
        10000,
        as_='cost'
    )
).transform_calculate(
    **calculations
).mark_bar().encode(
    y='rent:Q',
    x=alt.X('cost:Q', axis=alt.Axis(format='$,.0f', title='House Price')),
    tooltip=alt.Tooltip(['rent:Q', 'cost:Q']),
    color=alt.condition(
        datum.rent < MAX_COST,
        alt.value('blue'),
        alt.value('lightgray')
    )
)

# text = bars.mark_text(
#     align='right',
# #     baseline='middle',
#     dx=3  # Nudges text to right so it doesn't appear on top of the bar
# ).encode(
#     text='cost:Q'
# )

# bars = source
# bars
bars.add_selection(
    lease_term_selection
).add_selection(
    interest_selection
).add_selection(
    savings_selection
).add_selection(
    tax_selection
).add_selection(
    insurance_selection
).add_selection(
    utilities_selection
).add_selection(
    closing_fee_selection
).add_selection(
    rooms_selection
).properties(
    width=800,
    height=400
)
# bars

alt.Chart(...)